In [302]:
import pandas as pd
import numpy as np

In [303]:
da = pd.read_csv('data\itresume-coderun.csv', encoding='1251')

In [304]:
do = pd.read_csv('data\itresume-users-pandas.csv', encoding = '1251', sep = ';', skiprows = 1)

Загрузите файлы в датафреймы Pandas и назовите их coderun1 и users1. Далее создайте новый датафрейм df1_1, который получается в результате:
Объединения двух датафреймов по полям с пользователем
Если для каких-то кодранов не найдена пара в таблице users, такие записи нужно все равно оставить в результирующей таблице
Если есть совпадающие названия столбцов, то у них в результате должны быть суффиксы _run и _user

In [305]:
coderun1 = da.copy()
coderun1.head()

,id,created_at,problem_id,user_id,language_id
0,1,2021-04-07 06:06:20.000,13,10,3
1,2,2021-03-31 07:10:06.000,15,13,3
2,3,2021-04-04 14:55:26.000,1,6,3
3,4,2021-03-29 21:24:51.000,26,4,3
4,5,2021-03-30 11:29:12.000,21,18,3


In [306]:
users1 = do.copy()
users1.head()

,id,username,date_joined
0,416,nsosabisfsobm,2022-01-02 12:22:43
1,43,NbabnnbTona,2021-07-01 09:11:42
2,50,jbdn,2021-08-04 09:01:56
3,249,Kbaoxsnb,2021-11-27 07:03:35
4,77,bxakbasmo$$$,2021-11-09 14:31:46


In [307]:
df1_1 = coderun1.merge(users1,
				   left_on = 'user_id',
				   right_on = 'id', 
				   how = 'left',
				   suffixes = ['_run', '_user'])

In [308]:
df1_1

,id_run,created_at,problem_id,user_id,language_id,id_user,username,date_joined
0,1,2021-04-07 06:06:20.000,13,10,3,NaN,NaN,NaN
1,2,2021-03-31 07:10:06.000,15,13,3,NaN,NaN,NaN
2,3,2021-04-04 14:55:26.000,1,6,3,NaN,NaN,NaN
3,4,2021-03-29 21:24:51.000,26,4,3,NaN,NaN,NaN
4,5,2021-03-30 11:29:12.000,21,18,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
55717,55718,2022-05-17 09:00:51.743,101,171,2,NaN,NaN,NaN
55718,55719,2022-05-17 09:01:04.210,101,171,2,NaN,NaN,NaN
55719,55720,2022-05-17 09:02:06.203,101,171,2,NaN,NaN,NaN
55720,55721,2022-05-17 09:03:45.265,102,171,2,NaN,NaN,NaN


Создайте новый датафрейм df1_2, который будет хранить информацию о том, сколько дней прошло между последним выполнением кода конкретного юзера и днем его регистрации. Столбец с разницей должен называться diff.
Округлите diff до ближайшего меньшего числа, кратного 50. Результат должен остаться целым числом.

In [309]:
df1_2 = df1_1.copy()

In [310]:
df1_2 = (df1_2.groupby('user_id')
         .apply(lambda x: pd.Series({
             'diff': (max(pd.to_datetime(x['created_at'])) - min(pd.to_datetime(x['date_joined']))).days
         }))
        ).reset_index()

C:\Users\avtuh\AppData\Local\Temp\ipykernel_25684\1581937453.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [311]:
df1_2['diff'] = ((df1_2['diff'] / 50).apply(np.floor)*50).astype("Int64")

In [312]:
df1_2

,user_id,diff
0,1,<NA>
1,2,<NA>
2,3,<NA>
3,4,<NA>
4,5,<NA>
...,...,...
863,2804,<NA>
864,2812,<NA>
865,2813,<NA>
866,2814,<NA>


Создайте датафрейм df1_3, в который запишите количество юзеров, сгруппированных по разнице дней.
Результат отсортируйте по убыванию количества user_id.

In [313]:
df1_3 = (df1_2.groupby('diff', dropna=False)
		 .agg({'user_id' : 'count'})
		 .reset_index()
		 .sort_values(by = 'user_id', ascending=False))

In [314]:
df1_3

,diff,user_id
5,<NA>,795
0,0,41
2,100,16
1,50,11
3,150,3
4,250,2


Загрузите файл в датафрейм Pandas и назовите его coderun2. Создайте в этом датафрейме столбец с датой и назовите его date.
Далее создайте новый датафрейм df2_1, который получается в результате:
отсечения всех дат до 01.01.2022
группировки по дате
расчета количества задач, решаемых в каждый день (столбец назовите problems)
«превращение» столбца с датой из индекса в обычный столбец

In [315]:
coderun2 = da.copy()
coderun2.head()

,id,created_at,problem_id,user_id,language_id
0,1,2021-04-07 06:06:20.000,13,10,3
1,2,2021-03-31 07:10:06.000,15,13,3
2,3,2021-04-04 14:55:26.000,1,6,3
3,4,2021-03-29 21:24:51.000,26,4,3
4,5,2021-03-30 11:29:12.000,21,18,3


In [ ]:
coderun2['date'] = pd.to_datetime(coderun2['created_at']).dt.date

coderun2

,id,created_at,problem_id,user_id,language_id,date
0,1,2021-04-07 06:06:20.000,13,10,3,2021-04-07
1,2,2021-03-31 07:10:06.000,15,13,3,2021-03-31
2,3,2021-04-04 14:55:26.000,1,6,3,2021-04-04
3,4,2021-03-29 21:24:51.000,26,4,3,2021-03-29
4,5,2021-03-30 11:29:12.000,21,18,3,2021-03-30
...,...,...,...,...,...,...
55717,55718,2022-05-17 09:00:51.743,101,171,2,2022-05-17
55718,55719,2022-05-17 09:01:04.210,101,171,2,2022-05-17
55719,55720,2022-05-17 09:02:06.203,101,171,2,2022-05-17
55720,55721,2022-05-17 09:03:45.265,102,171,2,2022-05-17


In [318]:
start_date = pd.Timestamp('2022-01-01')

In [326]:
df2_1 = coderun2[pd.to_datetime(coderun2['date']) >= start_date].groupby('date')['problem_id'].agg(problems = 'count').reset_index()

df2_1

,date,problems
0,2022-01-01,83
1,2022-01-02,26
2,2022-01-03,55
3,2022-01-04,302
4,2022-01-05,503
...,...,...
131,2022-05-13,369
132,2022-05-14,201
133,2022-05-15,195
134,2022-05-16,397


Создайте столбец rolling_avg, который получается в результате усреднения столбца problems по 7 соседним измерениям. При этом усреднение происходит не по последним 7, а по:
предыдущим 3 элементам
текущему элементу
следующим 3 элементам
Возникшие 

In [327]:
df2_1['rolling_avg'] = df2_1['problems'].rolling(7, center = True).mean().fillna(0).round(2)

In [328]:
df2_1

,date,problems,rolling_avg
0,2022-01-01,83,0.00
1,2022-01-02,26,0.00
2,2022-01-03,55,0.00
3,2022-01-04,302,185.86
4,2022-01-05,503,218.43
...,...,...,...
131,2022-05-13,369,292.71
132,2022-05-14,201,272.43
133,2022-05-15,195,0.00
134,2022-05-16,397,0.00
